# 1. Frame the problem
Using the customer description, define the problem you're trying to solve in your own words (remember this is not technical but must be specific so the customer understands the project).

The goal of the project is to make a model that predicts the best rotation and drop spot for the piece in a tetris game in order to maximize the number of peices dropped before dying. The performence measure will be the number of moves made before dying but lines cleared will also be taken into account. Since the game isn't keeping track of any sort of score other than lines cleared and moves made, nothing else will have to be considered, unlike some verisonss of tetris. Additionally, the peices given are compeltely random.

# 2. Get the Data 
Define how you recieved the data (provided, gathered..)

Data was provided by the client. I cloned it into jupyter notbeook for this project.


In this case the data was the game which was given by the client.

# 3. Explore the Data
Gain insights into the data you have from step 2, making sure to identify any bias

I'm going to explore the data through playing the game. Here's what I saw:

- the Game's main focus is to survive, there is no score being kept
- there's a bug with peices being dropped in the corner (this was fixed)
- The output of the peices seems to be random, and there are 6 different possible peices (confirmed through peice.py)
-
I also looked at board and piece to find what I would have to change and saw the shape of the pieces.
- the piece would haved to be turned into something that could be inputed.
- get_next_rotation is what rotates the peice and place is what drops it


# 4.Prepare the Data


Apply any data transformations and explain what and why


The piece needed to be turned into an input that could be used along with the board for a neural network so I tried to linearize it. When that didn't work as intented, I manually one-hot encoded it and added it to piece.py as a function.

No other data

# 5. Model the data
Using selected ML models, experment with your choices and describe your findings. Finish by selecting a model to continue with


In [ ]:
# The first thing I tried was a DQN
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

class CUSTOM_AI_MODEL(nn.Module):
    def __init__(self):
        super(CUSTOM_AI_MODEL, self).__init__()
        self.fc1 = nn.Linear(247, 256)
        self.fc2 = nn.Linear(256, 256)
        self.fc3 = nn.Linear(256, 247)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def get_best_move(self, board, piece):
        """
        Computes the best move using the DQN model and evaluates its reward.
        """
        # Preprocess board and piece as input state
        linearized_board = np.array(board.board).flatten()
        one_hot_piece = np.array(piece.enumerate())
        state = np.concatenate((linearized_board, one_hot_piece))

        state_tensor = torch.FloatTensor(state).unsqueeze(0)

        # Predicts Q-values for all actions
        with torch.no_grad():
            q_values = self.forward(state_tensor)

        # Chooses the best action
        best_action = torch.argmax(q_values).item()
        best_rotation = best_action // board.width  # Rotation
        best_x = best_action % board.width         # Column

        # Apply rotation
        for _ in range(best_rotation):
            piece = piece.get_next_rotation()

        # Simulate the board state after the action
        new_board, rows_cleared, gaps, height = self.simulate_board(
            board, piece, best_x
        )

        # Compute the reward
        reward = self.compute_reward(rows_cleared, gaps, height)

        print(f"Move: {best_x}, Reward: {reward}")
        return best_x, piece

    def simulate_board(self, board, piece, x):
        """
        Simulates the board state after dropping the piece at column `x`.
        This method creates a deep copy of the board state manually.
        """
        # Manually create a copy of the board's state
        simulated_board = np.copy(board.board)
    
        # Simulate the dropping of the piece
        y = board.drop_height(piece, x)
        simulated_board = board.place_piece(piece, x, y)
    
        # Measure rows cleared
        rows_cleared = board.clear_rows()
    
        # Measure gaps (holes) in the board
        gaps = self.count_gaps(simulated_board)
    
        # Measure height of the highest stack
        height = self.max_stack_height(simulated_board)
    
        return simulated_board, rows_cleared, gaps, height


    def count_gaps(self, board):
        """
        Counts the number of gaps (empty cells with a block above them).
        """
        gaps = 0
        for col in range(board.width):
            found_block = False
            for row in range(board.height):
                if board[row][col] != 0:
                    found_block = True
                elif found_block:
                    gaps += 1
        return gaps

    def max_stack_height(self, board):
        """
        Returns the height of the tallest stack on the board.
        """
        for row in range(board.height):
            if any(board[row][col] != 0 for col in range(board.width)):
                return board.height - row
        return 0

    def compute_reward(self, rows_cleared, gaps, height):
        """
        Computes the reward based on rows cleared, gaps, and stack height.
        """
        reward = 0
        reward += rows_cleared * 10  # Positive reward for clearing rows
        reward -= gaps * 2           # Penalize for holes
        reward -= height             # Penalize for higher stacks
        return reward


I tried reinforcement learning at first through a Deep-Q-Network, which combines convultional neural netowrks and Q-learning. I punished it for holes and peaks. This code lasted about 10 moves on a average without any training. (A bit worse than the random)

Next I wanted to try using a weighted sum like that in the genetic algorithm, which I would train to find the best weights. I kept the peaks and the holes (remmade the functions though) and also checked for rowss filled and the evenness of the board. Since I wanted to make everything a cost, i changed the evenness to bumpiness to be punished instead of rewarded like evenness.

In [11]:
# Here is the untrained code using random weights between 0 and -1.
from copy import deepcopy
import numpy as np
import random

class CUSTOM_AI_MODEL:
    def __init__(self, weights=None, nfeatures=4, mutate=False, noise=0.01):
        self.weights = weights
        self.nfeatures = nfeatures
        self.mutate = mutate
        self.noise = noise

        if self.weights is None:
            # Default weights, updated to include the "holes" feature
            self.weights = np.array([random.uniform(-1, 0) for feature in range(nfeatures)])
            # self.weights = np.array([-1, -1, -1, -1]) # -1 version
        elif mutate:
            self.weights = weights * (np.array([np.random.normal(1, noise) for _ in range(nfeatures)]))

        self.fit_score = 0.0  # Fitness score
        self.fit_rel = 0.0  # Relative fitness compared to other agents

    def __lt__(self, other):
        return self.fit_score < other.fit_score

    def how_good(self, board):
        npeaks = peaks(board)
        nbumpiness = bumpiness(npeaks)
        nholes = sum(get_holes(npeaks, board))  # Total number of holes across all columns

        # Feature vector includes peaks, bumpiness, filled rows, and holes
        ratings = np.array([
            np.sum(npeaks),  # Total peaks
            nbumpiness,  # Bumpiness
            np.count_nonzero(np.mean(board, axis=1)),  # Partially filled rows
            nholes  # Total number of holes
        ])

        # Score the board state by taking the dot product of features and weights
        return np.dot(ratings, self.weights)

    def get_best_move(self, board, piece):
        best_x = -1000
        max_value = -1000
        best_piece = None

        for i in range(4):  # Loop through all possible rotations
            piece = piece.get_next_rotation()
            for x in range(board.width):  # Loop through all possible placements
                try:
                    y = board.drop_height(piece, x)
                except:
                    continue

                # Create a copy of the board and simulate the piece placement
                board_copy = deepcopy(board.board)
                for pos in piece.body:
                    board_copy[y + pos[1]][x + pos[0]] = True

                # Convert board_copy to a numpy array of 0s and 1s
                board_copy = np.asarray([[1 if cell else 0 for cell in row] for row in board_copy])
                c = self.how_good(board_copy)

                # Update the best move if this placement has a higher score
                if c > max_value:
                    max_value = c
                    best_x = x
                    best_piece = piece

        return best_x, best_piece

# New Function: get_holes
def get_holes(peaks, area):
    """
    Count the number of empty cells (holes) below the peaks in each column.
    """
    holes = []
    for col in range(area.shape[1]):
        start = -peaks[col]  # Topmost filled block in the column
        if start == 0:  # No blocks in the column
            holes.append(0)
        else:
            holes.append(np.count_nonzero(area[int(start):, col] == 0))
    return holes

def peaks(board):
    """
    Calculate the height of the tallest block in each column of the board.
    """
    peaks = np.array([]) 
    nrow, ncol = board.shape[1], board.shape[0]

    for col in range(nrow):
        if 1 in board[:, col]:  
            k = ncol - np.argmax(board[:, col][::-1], axis=0)
            peaks = np.append(peaks, k)
        else:
            peaks = np.append(peaks, 0)
    return peaks

def bumpiness(npeaks):
    """
    Calculate the total bumpiness (differences in adjacent column heights).
    """
    total = 0
    for i in range(len(npeaks) - 1):
        total += np.abs(npeaks[i] - npeaks[i + 1])
    return total

The -1 weight on everything perfemed between 52 and 400 something lines cleared, so I decided to move forward with this model.

# 6. Fine Tune the Model

With the select model descibe the steps taken to acheve the best results possible 


In [ ]:
# I used the following code to train the model
from os import pardir
import numpy as np
from game import Game
from custom_model import CUSTOM_AI_MODEL
import random
import pandas as pd
import pickle

def cross(a1, a2):

    new_weights = []
    
    prop = a1.fit_rel / a2.fit_rel
    
    for i in range(len(a1.weights)):
        rand = random.uniform(0, 1)
        if rand > prop:
            new_weights.append(a1.weights[i])
        else:
            new_weights.append(a2.weights[i])

    return CUSTOM_AI_MODEL(weights = np.array(new_weights), mutate = True)

def compute_fitness(agent, trials):

    fitness = []
    
    for i in range(trials):
        game = Game('student', agent = agent)
        peices_dropped, rows_cleared = game.run_no_visual()
        fitness.append(peices_dropped)
     
    return np.average(np.array(fitness))

def run_X_epochs(num_epochs=10, num_trials=20, pop_size=100, num_elite=10, survival_rate=0.2):
    data = []
    population = [CUSTOM_AI_MODEL() for _ in range(pop_size)]

    for epoch in range(num_epochs):
        total_fitness = 0
        top_agent = None

        # Evaluate each agent
        for agent in population:
            agent.fit_score = compute_fitness(agent, trials=num_trials)
            total_fitness += agent.fit_score

        # Normalize fitness scores
        for agent in population:
            agent.fit_rel = agent.fit_score / total_fitness

        # Sort population by fitness
        sorted_pop = sorted(population, reverse=True)
        next_gen = []

        # Retain elite agents
        for i in range(num_elite):
            next_gen.append(CUSTOM_AI_MODEL(weights=sorted_pop[i].weights, mutate=False))

        # Select parents and create offspring
        num_parents = round(pop_size * survival_rate)
        parents = sorted_pop[:num_parents]
        for _ in range(pop_size - num_elite):
            parent1, parent2 = random.sample(parents, 2)
            next_gen.append(cross(parent1, parent2))

        # Replace population with next generation
        population = next_gen

        # Log best agent and weights
        top_agent = sorted_pop[0]
        data.append(top_agent.weights)
        print(f"Epoch {epoch + 1}: Best Fitness = {top_agent.fit_score}")

        # Save data after each epoch
        with open("data.txt", "wb") as f:
            pickle.dump(data, f)

    return data

run_X_epochs()


# Here are the weights I decided to use: [-1.93725271, -0.29199838, -0.07409222, -0.37546867]

# 7. Present
In a customer facing document provide summary of finding and detail the approach taken


The first thing I did was assess the goal, which was to survive a game of tetris. The accuracy measure is the number of pieces able to be dropped before reaching the top and losing the game. The customer provided the data, and I flattened the board and one-hot encoded the piece to make it input-able for a neural network.

Then I tried a DQN and a weighted sum that would be genetically trained. The sum performed better, and although reinforcement learning would’ve probably been the most accurate long-term, I decided to move ahead with the latter. 

I trained the model genetically, so a random weight for each object is chosen and ran (each instance of weights is an agent) and the best performing agents were combined in crossover, with a little randomness introduced in mutation. I ran 20 epochs and since the values in training wouldn’t be that similar to testing due to the randomness of pieces given I manually looked at them to decide on the weights.

Some possible bias includes choosing what I considered, (evenness, peaks, rows filled, and holes), the survival rate (20% would “reproduce”) and a lot of the inputs in training. Additionally, in training I set each weight to a value between 0 and -1, and the randomness and lack of positive values probably both had an impact on the final model.

One issue that I couldn’t identify the cause of was that during training the models seemed to be doing well reaching over 1000, and even over 2000 once, but the most they reached in testing was a couple a hundred.

# 8. Launch the Model System
Define your production run code; this should be self sufficient and require only your model parameters 


This code should be launched through running the main file in terminal with student.